In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.config import SAMPLE_RATE
from noteify.core.datasets import (MusicNetDataset, MusicNetDatasetProcessed, MusicAugmentor,
                                   MusicSegmentSampler, get_music_dataloader)
from noteify.core.training import make_optimizer, make_scheduler, train_model
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll_info
from noteify.utils import get_rel_pkg_path

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

In [ ]:
data_dir = get_rel_pkg_path("musicnet/")
save_dir = get_rel_pkg_path("weights/")

In [ ]:
raw_dataset_train = MusicNetDataset(data_dir, download=True, train=True,
                                    numpy_cache=True, piano_only=True) #, filter_records=[1727])
raw_dataset_test = MusicNetDataset(data_dir, download=True, train=False,
                                   numpy_cache=True, piano_only=True) #, filter_records=[1759])

In [ ]:
batch_size = 32

In [ ]:
dataset_train = MusicNetDatasetProcessed(raw_dataset_train, augmentor=MusicAugmentor())
sampler_train = MusicSegmentSampler(dataset_train, batch_size, num_batches=None, random_start_times=True)
workers = 3 if raw_dataset_train.piano_only else 2
dataloader_train = get_music_dataloader(dataset_train, sampler_train, num_workers=workers, pin_memory=True)

dataset_test = MusicNetDatasetProcessed(raw_dataset_test)
sampler_test = MusicSegmentSampler(dataset_test, batch_size)
dataloader_test = get_music_dataloader(dataset_test, sampler_test, num_workers=None, pin_memory=True)

dataloaders = {'train': dataloader_train, 'test': dataloader_test}

In [ ]:
model = TranscriptionNN()
model = model.to(device)
past_train_weights = os.path.abspath(os.path.join(save_dir, "Experiment 12-08-2020 02-50AM/Weights Best.pckl"))
model.load_state_dict(torch.load(past_train_weights))

In [ ]:
optimizer = make_optimizer(model, lr=0.001)

In [ ]:
num_epochs = 100
scheduler = make_scheduler(optimizer, [60], gamma=0.5)

In [ ]:
tracker = train_model(device=device,
                      model=model,
                      dataloaders=dataloaders,
                      optimizer=optimizer,
                      lr_scheduler=scheduler,
                      save_model=True,
                      save_dir=save_dir,
                      save_best=True,
                      save_all=False,
                      num_epochs=num_epochs)

In [ ]:
%qtconsole